In [1]:
import subprocess
import shlex
import pathlib

In [2]:
def delete_generated_files(exclude:list = []):
    for f in pathlib.Path(".").iterdir():
        if not (f.name.endswith(".ipynb") or f.name in exclude):
            f.unlink(missing_ok=True)

delete_generated_files()

In [3]:
def print_message(process:subprocess.CompletedProcess,
                  success_msg:str = "success",
                  failed_msg:str = "failed"):
    if process.returncode == 0 :
        print(f"\N{White Heavy Check Mark}{success_msg} ")
    else:
        print(f"\N{Cross Mark} {failed_msg}")

# generate dependency with system header file (-M)

In [4]:
%%writefile dummy.cc
int main(){
    return 0;
}

Writing dummy.cc


In [5]:
gen_depend = subprocess.run(shlex.split("g++ -M dummy.cc -o dummy.pic.d"))
print_message(gen_depend,"Generated Dependency Succeed","Generated Dependency Failed")

✅Generated Dependency Succeed 


In [6]:
with open("dummy.pic.d" ,"r") as f:
    print(*f.readlines())

dummy.o: dummy.cc /usr/include/stdc-predef.h



In [7]:
%%writefile dummy1.cc
#include <iostream>
int main(){
    std::cout << "hello" << std::endl;
    return 0;
}

Writing dummy1.cc


In [8]:
gen_depend = subprocess.run(shlex.split("g++ -M dummy1.cc -o dummy1.pic.d"))
print_message(gen_depend,"Generated Dependency Succeed","Generated Dependency Failed")

✅Generated Dependency Succeed 


In [9]:
with open("dummy1.pic.d" ,"r") as f:
    print(*f.readlines())

dummy1.o: dummy1.cc /usr/include/stdc-predef.h \
  /usr/include/c++/11/iostream \
  /usr/include/x86_64-linux-gnu/c++/11/bits/c++config.h \
  /usr/include/x86_64-linux-gnu/c++/11/bits/os_defines.h \
  /usr/include/features.h /usr/include/features-time64.h \
  /usr/include/x86_64-linux-gnu/bits/wordsize.h \
  /usr/include/x86_64-linux-gnu/bits/timesize.h \
  /usr/include/x86_64-linux-gnu/sys/cdefs.h \
  /usr/include/x86_64-linux-gnu/bits/long-double.h \
  /usr/include/x86_64-linux-gnu/gnu/stubs.h \
  /usr/include/x86_64-linux-gnu/gnu/stubs-64.h \
  /usr/include/x86_64-linux-gnu/c++/11/bits/cpu_defines.h \
  /usr/include/c++/11/pstl/pstl_config.h /usr/include/c++/11/ostream \
  /usr/include/c++/11/ios /usr/include/c++/11/iosfwd \
  /usr/include/c++/11/bits/stringfwd.h \
  /usr/include/c++/11/bits/memoryfwd.h /usr/include/c++/11/bits/postypes.h \
  /usr/include/c++/11/cwchar /usr/include/wchar.h \
  /usr/include/x86_64-linux-gnu/bits/libc-header-start.h \
  /usr/include/x86_64-linux-gnu/b

# generate dependency without system header file (-MM)

In [10]:
gen_depend_ns = subprocess.run(shlex.split("g++ -MM dummy1.cc -o dummy1_ns.pic.d"))
print_message(gen_depend_ns,"Generated Dependency Succeed","Generated Dependency Failed")

✅Generated Dependency Succeed 


In [11]:
with open("dummy1_ns.pic.d" ,"r") as f:
    print(*f.readlines())

dummy1.o: dummy1.cc



# generate and compile (-MD)

Till now we only generated the dependency not compile.

In [12]:
subprocess.run(["ls -lha . | grep dummy1"],shell=True)

-rw-r--r-- 1 root root   87 Feb  5 14:17 dummy1.cc
-rw-r--r-- 1 root root 7.8K Feb  5 14:17 dummy1.pic.d
-rw-r--r-- 1 root root   20 Feb  5 14:17 dummy1_ns.pic.d


CompletedProcess(args=['ls -lha . | grep dummy1'], returncode=0)

In [13]:
gen_depend_ns_and_compile = subprocess.run(shlex.split("g++ -MD dummy1.cc -o dummy1.o"),check=True)
print_message(gen_depend_ns_and_compile,"Generated Dependency Succeed","Generated Dependency Failed")

✅Generated Dependency Succeed 


In [14]:
subprocess.run(["ls -lha . | grep dummy1"],shell=True)

-rw-r--r-- 1 root root   87 Feb  5 14:17 dummy1.cc
-rw-r--r-- 1 root root 7.8K Feb  5 14:17 dummy1.d
-rwxr-xr-x 1 root root  17K Feb  5 14:17 dummy1.o
-rw-r--r-- 1 root root 7.8K Feb  5 14:17 dummy1.pic.d
-rw-r--r-- 1 root root   20 Feb  5 14:17 dummy1_ns.pic.d


CompletedProcess(args=['ls -lha . | grep dummy1'], returncode=0)

In [15]:
delete_generated_files()